In [1]:
from pyspark.sql.functions import count
import pandas as pd
from lib import common_functions
from lib import configuration

In [ ]:
spark = common_functions.get_spark_session('player_stats')
spark.active()

In [ ]:
common_functions.make_dir(configuration.football_players_input_path)
common_functions.make_dir(configuration.football_players_output_path)

/home/jovyan/code/files/input/football/player_stats/ - Folder Already Exists!
/home/jovyan/code/files/output/football/player_stats/ - Folder Already Exists!


In [4]:
player_stats = spark.read.option("header", True).load(f'{configuration.football_players_input_path}/*.csv', format='csv')
player_stats.show()
player_stats.count()

+-----------------+-------+-----------+-----------+----------+-----------+--------+-----------+-------+------------+-------------+-----------+---------------+------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|           player|   team|       comp|appearences|  position|shots_total|shots_on|goals_total|assists|passes_total|tackles_total|fouls_drawn|fouls_committed|cards_yellow|    shots_accuracy|   shots_on_per_app|      goals_per_app|    assists_per_app|    passes_per_app|    tackles_per_app|      fouls_per_app|      cards_per_app|fouls_drawn_per_app|
+-----------------+-------+-----------+-----------+----------+-----------+--------+-----------+-------+------------+-------------+-----------+---------------+------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-----------

194

In [5]:
individual_team_home = 'Rangers'
individual_team_away = 'Celtic'

Filter to make new DF's

In [6]:
shots_on = player_stats.select("player", "team", "appearences", "shots_total", "shots_on", "shots_accuracy", "shots_on_per_app", "goals_total", "assists").where(    (player_stats['appearences']>5) & (player_stats['shots_on_per_app']>0.9) & (player_stats.shots_total / player_stats.appearences>1.7)    ) \
    .withColumn("shots_total_per_app", player_stats.shots_total / player_stats.appearences)
shots_on.sort("shots_total_per_app", ascending=[False]).show()

shots_on.count()

+------------+-------------------+-----------+-----------+--------+-----------------+------------------+-----------+-------+-------------------+
|      player|               team|appearences|shots_total|shots_on|   shots_accuracy|  shots_on_per_app|goals_total|assists|shots_total_per_app|
+------------+-------------------+-----------+-----------+--------+-----------------+------------------+-----------+-------+-------------------+
|  E. Haaland|    Manchester City|         15|         58|    38.0|65.51724137931035| 2.533333333333333|         13|      1| 3.8666666666666667|
|   K. Piątek|Istanbul Basaksehir|         14|         38|    20.0|52.63157894736842|1.4285714285714286|          9|      1| 2.7142857142857144|
|   C. Palmer|            Chelsea|         15|       37.0|    24.0|64.86486486486487|               1.6|         11|      6|  2.466666666666667|
|K. Furuhashi|             Celtic|         14|       34.0|    24.0|70.58823529411765|1.7142857142857142|          6|      3| 2.428

14

In [7]:
fouls_per_app = player_stats.select("player", "team", "appearences", "tackles_total", "tackles_per_app", "fouls_per_app", "cards_per_app").where(   (player_stats['appearences']>5) & (player_stats['tackles_per_app']>2.0) & (player_stats['fouls_per_app']>1.4)   )
#fouls_per_app.sort("fouls_per_app", ascending=[False]).show()
fouls_per_app.sort("tackles_per_app", ascending=[False]).show()

fouls_per_app.count()

+-------------+-------------------+-----------+-------------+------------------+------------------+-------------------+
|       player|               team|appearences|tackles_total|   tackles_per_app|     fouls_per_app|      cards_per_app|
+-------------+-------------------+-----------+-------------+------------------+------------------+-------------------+
|   T. Bischof|    1899 Hoffenheim|         11|           49| 4.454545454545454|1.4545454545454546|0.18181818181818182|
|   M. Caicedo|            Chelsea|         15|           47|3.1333333333333333|               2.0|                0.2|
|  C. Nørgaard|          Brentford|         11|           34| 3.090909090909091|1.4545454545454546|0.18181818181818182|
|    M. Ugarte|  Manchester United|         10|           25|               2.5|               1.5|                0.3|
|Miguel Crespo|Istanbul Basaksehir|         11|         26.0|2.3636363636363638|1.8181818181818181|0.18181818181818182|
|Emerson Royal|           AC Milan|     

7

In [8]:
fouls_drawn_per_app = player_stats.select("player", "team", "appearences", "tackles_total", "fouls_drawn_per_app", "cards_per_app").where(   (player_stats['fouls_drawn_per_app']>2.0) & (player_stats['appearences']>5)   )
fouls_drawn_per_app.sort("fouls_drawn_per_app", ascending=[False]).show()

fouls_drawn_per_app.count()

+-----------+-----------------+-----------+-------------+-------------------+-------------+
|     player|             team|appearences|tackles_total|fouls_drawn_per_app|cards_per_app|
+-----------+-----------------+-----------+-------------+-------------------+-------------+
|  C. Palmer|          Chelsea|         15|           10| 2.3333333333333335|          0.2|
|S. Guirassy|Borussia Dortmund|         10|          3.0|                2.3|          0.0|
+-----------+-----------------+-----------+-------------+-------------------+-------------+



2

In [9]:
individual_team_shots_on = shots_on.select(shots_on["*"]).where(    (shots_on["team"]==individual_team_home) | (shots_on["team"]==individual_team_away)   )
individual_team_shots_on.sort("shots_on_per_app", ascending=[False]).show()

+------------+-------+-----------+-----------+--------+-----------------+------------------+-----------+-------+-------------------+
|      player|   team|appearences|shots_total|shots_on|   shots_accuracy|  shots_on_per_app|goals_total|assists|shots_total_per_app|
+------------+-------+-----------+-----------+--------+-----------------+------------------+-----------+-------+-------------------+
|K. Furuhashi| Celtic|         14|       34.0|    24.0|70.58823529411765|1.7142857142857142|          6|      3| 2.4285714285714284|
|      Danilo|Rangers|          6|         14|    10.0|71.42857142857143|1.6666666666666667|          3|      1| 2.3333333333333335|
|     A. Idah| Celtic|         12|       23.0|    16.0|69.56521739130434|1.3333333333333333|          5|      0| 1.9166666666666667|
+------------+-------+-----------+-----------+--------+-----------------+------------------+-----------+-------+-------------------+



In [10]:
individual_team_fouls_per_app = fouls_per_app.select(fouls_per_app["*"]).where(    (fouls_per_app["team"]==individual_team_home) | (fouls_per_app["team"]==individual_team_away)   )
individual_team_fouls_per_app.sort("fouls_per_app", ascending=[False]).show()

#individual_team_fouls_per_app.toPandas().to_csv(f'{configuration.football_players_output_path}liv_man_city.csv')


+------+----+-----------+-------------+---------------+-------------+-------------+
|player|team|appearences|tackles_total|tackles_per_app|fouls_per_app|cards_per_app|
+------+----+-----------+-------------+---------------+-------------+-------------+
+------+----+-----------+-------------+---------------+-------------+-------------+



In [11]:
individual_team_fouls_drawn_per_app = fouls_drawn_per_app.select(fouls_drawn_per_app["*"]).where(    (fouls_drawn_per_app["team"]==individual_team_home) | (fouls_drawn_per_app["team"]==individual_team_away)   )
individual_team_fouls_drawn_per_app.sort("fouls_drawn_per_app", ascending=[False]).show()

+------+----+-----------+-------------+-------------------+-------------+
|player|team|appearences|tackles_total|fouls_drawn_per_app|cards_per_app|
+------+----+-----------+-------------+-------------------+-------------+
+------+----+-----------+-------------+-------------------+-------------+



In [12]:
team_count = fouls_per_app.groupBy("team").agg(count("*")).alias("count").sort("count(1)", ascending=[False])
fouls_per_app.groupBy("team").agg(count("*")).alias("count").sort("count(1)", ascending=[False]).show()

+-------------------+--------+
|               team|count(1)|
+-------------------+--------+
|Istanbul Basaksehir|       1|
|            Chelsea|       1|
|  Manchester United|       1|
|          Brentford|       1|
|    1899 Hoffenheim|       1|
|           AC Milan|       1|
|              Genoa|       1|
+-------------------+--------+



In [13]:
with pd.ExcelWriter(f'{configuration.football_players_output_path}player_stats.xlsx') as writer:  
     shots_on.toPandas().to_excel(writer, sheet_name='shots_on', index=False)
     fouls_per_app.toPandas().to_excel(writer, sheet_name='fouls_per_app', index=False)
     fouls_drawn_per_app.toPandas().to_excel(writer, sheet_name='fouls_drawn_per_app', index=False)
     team_count.toPandas().to_excel(writer, sheet_name='team_count', index=False)
     individual_team_shots_on.toPandas().to_excel(writer, sheet_name='individual_team_shots_on', index=False)
     individual_team_fouls_per_app.toPandas().to_excel(writer, sheet_name='individual_team_fouls_per_app', index=False)